<font size=6>Downloading required libraries</font>

In [ ]:
!pip3 install -q numpy
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
!pip3 install torch torchinfo
!pip3 install torch tensorboard
!pip3 install -q pretty_midi
!pip3 install -q gensim
!pip3 install -q nltk

<font size=6>Imports</font>

In [ ]:
import torch
import os
import torch.nn as nn
import torch.utils.data as data
import time
import torch.nn.functional as F
import random
import copy
import numpy as np
import math
import matplotlib.pyplot as plt
from collections import defaultdict
from torchinfo import summary
from typing import Optional
import csv
import string
from pretty_midi import PrettyMIDI, Note
from sklearn.model_selection import train_test_split
import re
import gensim.downloader
import nltk
from nltk.corpus import stopwords
from collections import Counter
nltk.download('stopwords')
import pickle
from torch.utils.tensorboard import SummaryWriter
from collections import deque

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

print("Using torch", torch.__version__)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

<font size=6>Constants</font>

In [ ]:
LYRIC_TRAIN_SET_CSV_PATH: str = os.path.join(os.getcwd(), 'data', 'lyrics_train_set.csv')
LYRIC_TEST_SET_CSV_PATH: str = os.path.join(os.getcwd(), 'data', 'lyrics_test_set.csv')
MIDI_FILE_PATH: str = os.path.join(os.getcwd(), 'data', 'midi_files')
PICKLING_PATH: str = os.path.join(os.getcwd(), 'loaded_midi_files.pkl') # Path to save/load pickled MIDI files, for faster loading.
EPSILON: float = 1e-9
SEQUENCE_LENGTH: int = 10  # Number of words in the input sequence
BATCH_SIZE: int = 128
LSTM_LAYERS: int = 2
ATTENTION_LAYERS: int = 2
DROPOUT: float = 0.3
ATTENTION_DROPOUT: float = 0.2
RANDOM_LOADER_SEED: int = 42
VALIDATION_SPLIT: float = 0.1
LEARNING_RATE: float = 0.001
MAX_EPOCHS: int = 100
NUMBER_OF_EXTRACT_MIDI_FEATURES: int = 25
WORD_EMBEDDING_SIZE: int = 300
SIZE_OF_ARTIST_INDEX: int = 1
PATIENCE_FACTOR: float = 0.01
PATIENCE_EPOCHS: int = 10
UNK_ID: int = 0
MAX_LINE_LENGTH: int = SEQUENCE_LENGTH
MIN_LINE_LENGTH: int = SEQUENCE_LENGTH / 2
EOL_STRING: str = '<eol>'
UNK_STRING: str = '<unk>'
PAD_STRING: str = '<pad>'
EOS_STRING: str = '<eos>'
TOP_K_WORDS_TO_PREDICT: int = 20
MAX_SONG_LENGTH_WORDS: int = 80
HIDDEN_LAYER_DIM: int = 256
HIDDEN_LAYER_DIM_ATTENTION: int = 256
SEED = 42
VERBOSE: str = True
NOTE_SAMPLING_INTERVAL_SHORT: float = 0.10
NOTE_SAMPLING_INTERVAL_LONG: float = 0.50
RHYTHM_FEATURES: str = 'RHYTHM_FEATURES'
HARMONY_FEATURES: str = 'HARMONY_FEATURES'
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

<font size=6>Midi Feature extraction</font>

Auxlliary functions

In [ ]:
def most_common_time_signature(changes: tuple[int, int]) -> tuple[int, int]:
    if not changes: return (4, 4)
    pairs: list[tuple[int, int]] = [(ts.numerator, ts.denominator) for ts in changes]
    return Counter(pairs).most_common(1)[0][0]

In [ ]:
def get_duration_weighted_pitch_stats(notes: list[Note]) -> dict[str, float]:
    """
    Compute pitch statistics weighted by each note's duration.
    Returns a dict with:
      mean (duration-weighted),
      std  (duration-weighted),
      p10 / p50 / p90 (duration-weighted percentiles),
      ambitus = p90 - p10 (robust range).
    If `notes` is empty, returns safe defaults.
    """
    # Empty guard: nothing to measure → return neutral stats.
    if not notes:
        return dict(mean=0.0, std=0.0, p10=-1, p50=-1, p90=-1, ambitus=0.0)

    # Vectorize pitches as float for math (MIDI 0..127, but floats simplify ops).
    pitches: np.ndarray[np.float32] = np.fromiter((n.pitch for n in notes), dtype=np.float32)

    # Each note's weight = its duration in seconds; clamp tiny/negative to epsilon.
    weights: np.ndarray[np.float32] = np.fromiter((max(EPSILON, n.end - n.start) for n in notes), dtype=np.float32)
    total_weights: float = weights.sum()
    duration_weight_mean: float = float((weights * pitches).sum() / total_weights)
    duration_weighted_variance: float = float((weights * (pitches - duration_weight_mean) ** 2).sum() / total_weights)
    weighted_std: float = duration_weighted_variance ** 0.5

    # ---------- Duration-weighted percentiles ----------
    order: np.ndarray[np.int32] = np.argsort(pitches)
    ordered_pitches, ordered_weights = pitches[order], weights[order]
    cumulative_weight_sum: np.ndarray[np.float32] = np.cumsum(ordered_weights)

    # Weighted quantile: find the first index where cumulative weight crosses q%.
    def weighted_quantile(quantile: float) -> float:
        # Target cumulative weight at quantile q (0..100).
        target: float = (quantile / 100.0) * cumulative_weight_sum[-1]
        # Index where cumulative_weight_sum >= target; take leftmost to be consistent.
        idx: int = np.searchsorted(cumulative_weight_sum, target, side="left")
        return float(ordered_pitches[min(idx, len(ordered_pitches) - 1)])

    # 10th / 50th (median) / 90th percentiles, duration-weighted.
    percentile_10, percentile_50, percentile_90 = weighted_quantile(10), weighted_quantile(50), weighted_quantile(90)

    # Ambitus = robust spread (p90 - p10), less sensitive than raw max - min.
    return dict(mean=duration_weight_mean, 
                std=weighted_std,
                p10=percentile_10, 
                p50=percentile_50, 
                p90=percentile_90)


In [ ]:
def tempo_stats(midi: PrettyMIDI) -> tuple[float, float, int]:
    tempo_times, tempo_bpms = midi.get_tempo_changes()
    if len(tempo_bpms) == 0:
        tempo_times = np.array([0.0], dtype=np.float32)
        tempo_bpms  = np.array([midi.estimate_tempo()], dtype=np.float32)

    # duration-weighted mean tempo + dispersion proxy
    duration_sec = midi.get_end_time()
    segment_ends: np.ndarray[np.float32] = np.r_[tempo_times[1:], duration_sec]
    segment_durs: np.ndarray[np.float32] = np.maximum(EPSILON, segment_ends - tempo_times[:len(segment_ends)])
    tempo_mean: float = float(np.dot(tempo_bpms[:len(segment_durs)], segment_durs) / np.sum(segment_durs))
    tempo_std: float = float(np.std(np.repeat(
        tempo_bpms[:len(segment_durs)],
        np.maximum(1, (segment_durs/np.sum(segment_durs)*1000).astype(int))
    )))
    tempo_change_count: int = int(len(tempo_bpms))
    return tempo_mean, tempo_std, tempo_change_count

In [ ]:
def collect_melodic_material(midi: PrettyMIDI) -> tuple[list[Note], list[int]]:
    notes: list[Note] = []
    velocities: list[int] = []
    for inst in midi.instruments:
        if inst.is_drum or not inst.notes:
            continue
        notes.extend(inst.notes)
        velocities.extend([n.velocity for n in inst.notes])
    return notes, velocities

In [ ]:

def instrument_count_non_drum_with_notes(midi: PrettyMIDI) -> int:
    return sum(1 for inst in midi.instruments if not inst.is_drum and inst.notes)

In [ ]:
def note_duration_stats(notes: list[Note]) -> tuple[float, float, float, float, float, float]:
    if not notes:
        return 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    durs = np.array([max(EPSILON, n.end - n.start) for n in notes], dtype=np.float32)
    mean = float(durs.mean())
    std  = float(durs.std())
    rng  = float(durs.max() - durs.min())
    cv   = float(std / max(EPSILON, mean))
    p10  = float(np.percentile(durs, 10))
    p90  = float(np.percentile(durs, 90))
    return mean, std, rng, cv, p10, p90

In [ ]:
def ioi_stats(notes: list[Note]) -> tuple[float, float, float, float, float]:
    if not notes:
        return 0.0, 0.0, 0.0, 0.0, 0.0
    starts = np.array(sorted(n.start for n in notes), dtype=np.float32)
    if len(starts) < 2:
        return 0.0, 0.0, 0.0, 0.0, 0.0
    iois = np.diff(starts)
    iois = np.clip(iois, EPSILON, None)
    mean = float(iois.mean())
    std  = float(iois.std())
    cv   = float(std / max(EPSILON, mean))
    short_ratio = float((iois <= NOTE_SAMPLING_INTERVAL_SHORT).mean())
    long_ratio  = float((iois >= NOTE_SAMPLING_INTERVAL_LONG).mean())
    return mean, std, cv, short_ratio, long_ratio

In [ ]:
def rest_ratio_extraction(notes: list[Note], duration_sec: float) -> float:
    if not notes or duration_sec <= 0:
        return 0.0
    notes_sorted = sorted(notes, key=lambda n: n.start)
    gaps = []
    for a, b in zip(notes_sorted[:-1], notes_sorted[1:]):
        gap = max(0.0, b.start - a.end)
        if gap > 0:
            gaps.append(gap)
    return float(sum(gaps) / max(EPSILON, duration_sec))

In [ ]:
def velocity_stats(velocities: list[int]) -> tuple[float, float, float, float]:
    if not velocities:
        return 0.0, 0.0, 0.0, 0.0
    arr = np.array(velocities, dtype=np.float32)
    vmin, vmax = float(arr.min()), float(arr.max())
    vmean, vstd = float(arr.mean()), float(arr.std())
    vcv = float(vstd / max(EPSILON, vmean))
    return vmin, vmax, vmean, vstd if vmean == 0 else vcv  # keep both std and cv separately elsewhere if needed

In [ ]:
def velocity_min_max_mean_std_cv(velocities: list[int]) -> tuple[float, float, float, float, float]:
    if not velocities:
        return 0.0, 0.0, 0.0, 0.0, 0.0
    arr = np.array(velocities, dtype=np.float32)
    vmin = float(arr.min())
    vmax = float(arr.max())
    vmean = float(arr.mean())
    vstd  = float(arr.std())
    vcv   = float(vstd / max(EPSILON, vmean))
    return vmin, vmax, vmean, vstd, vcv

In [ ]:
def interval_stats(notes: list[Note]) -> tuple[float, float, float]:
    if not notes:
        return 0.0, 0.0, 0.0
    ns = sorted(notes, key=lambda n: n.start)
    if len(ns) < 2:
        return 0.0, 0.0, 0.0
    pitches = np.array([n.pitch for n in ns], dtype=np.float32)
    steps = np.diff(pitches)
    abs_steps = np.abs(steps)
    if abs_steps.size == 0:
        return 0.0, 0.0, 0.0
    avg_abs = float(abs_steps.mean())
    small_step_ratio = float((abs_steps <= 2).mean())
    large_leap_ratio = float((abs_steps >= 7).mean())
    return avg_abs, small_step_ratio, large_leap_ratio

In [ ]:
def pitch_time_slope(notes: list[Note]) -> float:
    if not notes:
        return 0.0
    ns = sorted(notes, key=lambda n: n.start)
    if len(ns) < 2:
        return 0.0
    t = np.array([n.start for n in ns], dtype=np.float32)
    p = np.array([n.pitch for n in ns], dtype=np.float32)
    t = (t - t.mean()) / (t.std() + EPSILON)
    p = (p - p.mean()) / (p.std() + EPSILON)
    return float(np.clip(np.corrcoef(t, p)[0,1], -1.0, 1.0))

In [ ]:
def distinct_pitch_and_repetition(notes: list[Note]) -> tuple[float, float]:
    if not notes:
        return 0.0, 0.0
    pitches = [n.pitch for n in notes]
    total = len(pitches)
    distinct_ratio = float(len(set(pitches)) / max(1, total))
    # repetition as mode frequency / total
    values, counts = np.unique(pitches, return_counts=True)
    repetition_ratio = float(counts.max() / total)
    return distinct_ratio, repetition_ratio

In [ ]:
def chroma_features(midi: PrettyMIDI) -> tuple[np.ndarray, float]:
    chroma = midi.get_pitch_class_histogram(use_duration=True)
    chroma = chroma / (np.sum(chroma) + EPSILON)
    # Shannon entropy in nats
    with np.errstate(divide='ignore', invalid='ignore'):
        entropy = -np.nansum(chroma * np.log(chroma + EPSILON))
    return chroma.astype(np.float32), float(entropy)

In [ ]:
def extract_category_rhythm_dynamics_midi_features(midi: PrettyMIDI) -> tuple[np.ndarray, list[str]]:
    notes, velocities = collect_melodic_material(midi=midi)
    duration_sec = midi.get_end_time()
    tempo_mean, tempo_std, tempo_change_count = tempo_stats(midi)
    ts_num, ts_den = most_common_time_signature(midi.time_signature_changes)
    instr_count = instrument_count_non_drum_with_notes(midi)
    note_density = float(len(notes) / max(EPSILON, duration_sec))

    ioi_mean, ioi_std, ioi_cv, ioi_short_ratio, ioi_long_ratio = ioi_stats(notes)
    rest_ratio = rest_ratio_extraction(notes, duration_sec)

    dur_mean, dur_std, dur_range, dur_cv, dur_p10, dur_p90 = note_duration_stats(notes)

    v_min, v_max, v_mean, v_std, v_cv = velocity_min_max_mean_std_cv(velocities)

    names = [
        "duration_sec",
        "tempo_mean_bpm",
        "tempo_std_bpm",
        "tempo_change_count",
        "time_sig_num",
        "time_sig_den",
        "instrument_count",
        "melody_note_density_per_sec",
        "ioi_mean_sec",
        "ioi_std_sec",
        "ioi_cv",
        "rest_ratio",
        "note_durations_mean",
        "note_durations_std",
        "note_durations_range",
        "note_durations_cv",
        "note_duration_p10",
        "note_duration_p90",
        "short_note_ratio_le_0p10s",
        "long_note_ratio_ge_0p50s",
        "velocity_min",
        "velocity_max",
        "velocity_mean",
        "velocity_std",
        "velocity_cv",
    ]
    vec = np.array([
        duration_sec,
        tempo_mean,
        tempo_std,
        tempo_change_count,
        ts_num,
        ts_den,
        instr_count,
        note_density,
        ioi_mean,
        ioi_std,
        ioi_cv,
        rest_ratio,
        dur_mean,
        dur_std,
        dur_range,
        dur_cv,
        dur_p10,
        dur_p90,
        ioi_short_ratio,
        ioi_long_ratio,
        v_min,
        v_max,
        v_mean,
        v_std,
        v_cv,
    ], dtype=np.float32)
    return vec, names

In [ ]:
def extract_category_pitch_harmony_midi_features(midi: PrettyMIDI) -> tuple[np.ndarray, list[str]]:
    notes, _  = collect_melodic_material(midi=midi)
    # duration-weighted pitch stats you already have
    dwp = get_duration_weighted_pitch_stats(notes)
    pitch_mean = float(dwp["mean"])
    pitch_std  = float(dwp["std"])
    pitch_p10  = float(dwp["p10"])
    pitch_p50  = float(dwp["p50"])
    pitch_p90  = float(dwp["p90"])
    ambitus    = float(pitch_p90 - pitch_p10)

    avg_abs_interval, small_step_ratio, large_leap_ratio = interval_stats(notes)
    slope = pitch_time_slope(notes)
    distinct_ratio, repetition_ratio = distinct_pitch_and_repetition(notes)

    chroma, chroma_entropy = chroma_features(midi)

    names = [
        "pitch_mean_dw",
        "pitch_std_dw",
        "pitch_p10_dw",
        "pitch_p50_dw",
        "pitch_p90_dw",
        "pitch_ambitus_p90_p10",
        "avg_abs_interval",
        "small_step_ratio_le_2st",
        "large_leap_ratio_ge_7st",
        "pitch_time_slope_corr",
        "distinct_pitch_ratio",
        "repetition_ratio_mode",
        "chroma_entropy",
    ] + [f"chroma_{i}" for i in range(12)]

    vec = np.array([
        pitch_mean,
        pitch_std,
        pitch_p10,
        pitch_p50,
        pitch_p90,
        ambitus,
        avg_abs_interval,
        small_step_ratio,
        large_leap_ratio,
        slope,
        distinct_ratio,
        repetition_ratio,
        chroma_entropy,
        *chroma.tolist(),  # 12 elements, each representing a pitch class.
    ], dtype=np.float32)

    return vec, names

<font size=6>Auxlilliary Data Structures</font>

Auxilliary functions for creation of word sequences and targets

In [ ]:
def create_word_sequences_with_targets(tokenized_lyrics: list[str], sequence_length: int = SEQUENCE_LENGTH) -> tuple[np.ndarray, np.ndarray]:
    """
    Given tokenized lyrics as a list of strings, create sequences of word indices and their corresponding target word indices.
    Each sequence is of length `sequence_length`, and the target_sequence is a list of the next words of the sequence 1 index higher..
    """
    sequences = []
    targets = []
    
    # Create sequences and targets
    for i in range(len(tokenized_lyrics) - sequence_length):
        seq = tokenized_lyrics[i:i + sequence_length] # Padding
        target_sequence = tokenized_lyrics[i + 1: i + sequence_length + 1]
        sequences.append(seq)
        targets.append(target_sequence)
    
    return sequences, targets

In [ ]:
class SongData:
    def __init__(self, song_data_cell: list[str] = None, midi_file: PrettyMIDI = None):
        if len(song_data_cell) != 3:
            raise ValueError("song_data_cell must have exactly three elements: [artist, title, lyrics]")
        self.artist = song_data_cell[0]
        self.title = song_data_cell[1]
        self.lyrics = song_data_cell[2]
        self.raw_midi_data = midi_file
        self._midi_harmony_and_pitch_features: Optional[dict[str, np.ndarray]] = None
        self._midi_rhythm_and_dynamic_features: Optional[dict[str, np.ndarray]] = None

    @property
    def midi_harmony_and_pitch_features(self):
        if self._midi_harmony_and_pitch_features is None:
            self._midi_harmony_and_pitch_features = extract_category_pitch_harmony_midi_features(self.raw_midi_data)
        return self._midi_harmony_and_pitch_features
    
    @property
    def midi_rhythm_and_dynamic_features(self):
        if self._midi_rhythm_and_dynamic_features is None:
            self._midi_rhythm_and_dynamic_features = extract_category_rhythm_dynamics_midi_features(self.raw_midi_data)
        return self._midi_rhythm_and_dynamic_features

<font size=6>Reading CSV files</font>

In [ ]:
class SongDataset(data.Dataset):
    def __init__(self,
                songs_data: list[SongData],
                word_embeddings: dict[str, np.ndarray],
                artist_to_index_data_set: dict[str, int],
                word_to_id_data_set: dict[str, int],
                midi_features_to_use: str):
        self.midi_features: list[np.ndarray] = list()
        self.artists: list[str] = list()
        self.sequence_artists: list[str] = list()
        self.word_sequences: list[str] = list()
        self.sequences_targets: list[str] = list()
        self.sequence_to_midi: list[int] = list() # Maps each sequence to its corresponding MIDI feature index
        self.sequence_to_artist: list[int] = list() # Maps each sequence to its corresponding artist embedding index
        self.word_embeddings: dict[str, np.ndarray] = word_embeddings
        self.artist_to_index: dict[str, int] = artist_to_index_data_set
        self.word_to_id: dict[str, int] = word_to_id_data_set
        # Instead of saving each sequence's MIDI features, we save the index of the MIDI features in the midi_features list to save space.
        for idx, song in enumerate(songs_data):
            sequences, targets = create_word_sequences_with_targets(song.lyrics)
            self.word_sequences.extend(sequences)
            self.sequences_targets.extend(targets)
            if midi_features_to_use == HARMONY_FEATURES:
                extracted_features, _ = song.midi_harmony_and_pitch_features
            elif midi_features_to_use == RHYTHM_FEATURES:
                extracted_features, _ = song.midi_rhythm_and_dynamic_features
            else:
                raise ValueError(f"Type of features extraction not supported: {midi_features_to_use}")
            self.midi_features.append(extracted_features) # Creates a mapping of the features to the sequences.
            self.sequence_artists.append(song.artist)
            self.sequence_to_midi.extend([idx] * len(sequences))
            self.sequence_to_artist.extend([idx] * len(sequences))
        print(f'Dataset with {midi_features_to_use} has: {len(self.word_sequences)} sequences and {len(self.sequences_targets)} targets')


    def word_vec(self, tok: str) -> np.ndarray:
        # helper to get word vector, or zeros if no embedding
        v = self.word_embeddings.get(tok)
        if v is None:
            sample = next(iter(self.word_embeddings.values())) # Get size.
            v = np.zeros_like(sample, dtype=np.float32)
        return v.astype(np.float32, copy=False)

    def __len__(self):
        return len(self.word_sequences)

    def __getitem__(self, idx: int):
        tokens = self.word_sequences[idx]                # list[str], len T
        target_tokens = self.sequences_targets[idx]      # list[str], len T
        midi = self.midi_features[self.sequence_to_midi[idx]].astype(np.float32, copy=False)
        artist_name = self.sequence_artists[self.sequence_to_artist[idx]]
        artist_idx = np.float32(self.artist_to_index[artist_name])

        emb = np.stack([self.word_vec(tok) for tok in tokens], axis=0).astype(np.float32, copy=False)     # [T,E]
        midi_b = np.broadcast_to(midi, (emb.shape[0], midi.shape[0])).astype(np.float32, copy=False)      # [T,M]
        artist_b = np.full((emb.shape[0], 1), artist_idx, dtype=np.float32)                               # [T,1]

        concatenated_features = np.concatenate((emb, midi_b, artist_b), axis=1).astype(np.float32, copy=False)                # [T,D]
        target_words = np.asarray([self.word_to_id.get(target_token, self.word_to_id.get(UNK_STRING, 0)) for target_token in target_tokens],
                   dtype=np.int64)
        return concatenated_features, target_words

In [ ]:
with open(LYRIC_TRAIN_SET_CSV_PATH, mode='r', encoding='utf-8') as train_file:
    reader = csv.reader(train_file)
    lyric_train_data = list(reader)

with open(LYRIC_TEST_SET_CSV_PATH, mode='r', encoding='utf-8') as test_file:
    reader = csv.reader(test_file)
    lyric_test_data = list(reader)

if len(lyric_train_data) < 1 or len(lyric_test_data) < 1:
    raise Exception("CSV files are empty or not found.")

<font size=6>Parsing CSV files</font>

In [ ]:
def clean_csv_data(raw_csv_data: list[list[str]]) -> list[tuple[str, str, list[str]]]:
    returned_cleaned_csv_data: list[tuple[str, str, list[str]]] = []
    for row in raw_csv_data:
        artist = row[0].strip()
        title_index = 1
        lyrics_index = 2
        while lyrics_index < len(row):
            title = row[title_index].strip()
            title = title.removesuffix('-2') # Remove '-2' suffix if present, relevant in 1 case.
            title = title.strip()
            lyrics = row[lyrics_index].strip()
            lyrics = lyrics.lower()
            lyrics = re.sub(f"[{re.escape('\'')}]", "", lyrics) # Removing apostrophes.
            lyrics = re.sub(f"[{re.escape('-')}]", " ", lyrics) # Removing hyphens.
            lyrics = re.sub(f"[{re.escape('&')}]", f" {'PLACEHOLDERSTRING'} ", lyrics) # Putting PLACEHOLDER to not be removed by string_punctuation.
            lyrics = re.sub(f"[{re.escape(string.punctuation)}]", "", lyrics) # Removing punctuation.
            lyrics = re.sub('PLACEHOLDERSTRING', EOL_STRING, lyrics) # Putting PLACEHOLDER to not be removed by string_punctuation.
            lyrics = lyrics.split(' ') # Tokenzing each word by space.
            lyrics = [word.strip() for word in lyrics if word] # Removing empty strings.
            lyrics.append(EOS_STRING) # Adding end of song token.
            if len(title) > 0 and len(lyrics) > 0:
                returned_cleaned_csv_data.append((artist, title, lyrics))
            title_index += 2
            lyrics_index += 2
    return returned_cleaned_csv_data

cleaned_lyric_train_data = clean_csv_data(lyric_train_data)
cleaned_lyric_test_data = clean_csv_data(lyric_test_data)

In [ ]:
# count the number of unique words in the lyrics
def get_word_frequencies(lyrics_data: list[tuple[str, str, list[str]]]) -> dict[str, int]:
    words_frequency = defaultdict(int)
    for _, _, lyrics in lyrics_data:
        for word in lyrics:
            words_frequency[word] += 1
    return words_frequency    
word_frequencies_training: dict[str, int] = get_word_frequencies(cleaned_lyric_train_data)
word_frequencies_test: dict[str, int] = get_word_frequencies(cleaned_lyric_test_data)
print(f"Number of unique words in training set: {len(word_frequencies_training)}")
print(f"Number of unique words in test set: {len(word_frequencies_test)}")

d_sorted_by_val = sorted(word_frequencies_training.items(), key=lambda kv: kv[1], reverse=True)
d_sorted_by_val[:10]

<font size=6>Reading MIDI files</font>

In [ ]:
def load_midi_files(midi_files_location: str, pickling_path: Optional[str] = None, failed_loads_path: Optional[str] = None) -> \
                    tuple[dict[str, dict[str, PrettyMIDI]], dict[str, set[str]]]: # artist -> title -> PrettyMIDI, failed loads[artist, song_set]
    failed_loads = dict()
    if failed_loads_path is not None and os.path.isfile(failed_loads_path):
        with open(failed_loads_path, "rb") as f:
            failed_loads = pickle.load(f)
        print(f"Loaded failed MIDI loads from pickled file {failed_loads_path}.")
    if pickling_path is not None and os.path.isfile(pickling_path):
        with open(pickling_path, "rb") as f:
            loaded_midi_files = pickle.load(f)
        print(f"Loaded MIDI files from pickled file {pickling_path}.")
        print(f'Loaded {sum([len(songs) for songs in loaded_midi_files.values()])} MIDI files.')
        return loaded_midi_files, failed_loads
    if not os.path.isdir(midi_files_location):
        raise ValueError(f"MIDI file path {midi_files_location} is not a valid directory.")

    # Traversing over all files and attempt to load them with pretty_midi:
    loaded_midi_files: dict[str, dict[str, PrettyMIDI]] = defaultdict(dict) # artist -> title -> PrettyMIDI
    failed_loads: dict[str, set[str]] = defaultdict(set)

    for file in os.listdir(midi_files_location):
        if file.endswith('.mid') or file.endswith('.midi'):
            file_path = os.path.join(midi_files_location, file)
            file = file.removesuffix('.mid')
            splitted_artist_and_title = file.split('_-_')
            artist = splitted_artist_and_title[0]
            title = splitted_artist_and_title[1]
            if len(splitted_artist_and_title) > 2:
                print(f"Warning: file {file} has more than one '_-_' separator, ignoring the rest after second \"_-_\".")
            artist = artist.replace('_', ' ').strip().lower()
            title = title.replace('_', ' ').strip().lower()
            try:
                midi_data = PrettyMIDI(file_path)
                loaded_midi_files[artist][title] = midi_data
            except Exception as e:
                print(f"Failed to load {file}: {e}")
                failed_loads[artist].add(title)



    if failed_loads:
        print("Failed to load the following artist and lyric midi files:")
        for artist, lyrics in failed_loads.items():
            print(f"{artist} - [{', '.join(lyrics)}]")

    if pickling_path is not None:
        with open(pickling_path, "wb") as f:
            pickle.dump(loaded_midi_files, f)
            print(f"Pickled loaded MIDI files to {pickling_path}.")
    if failed_loads_path is not None:
        with open(failed_loads_path, "wb") as f:
            pickle.dump(failed_loads, f)
            print(f"Pickled failed MIDI loads to {failed_loads_path}.")

    print(f"Successfully loaded {sum([len(songs) for songs in loaded_midi_files.values()])} MIDI files.")
    if failed_loads:
        print("Failed to load the following midi files:")
        for failed_load in failed_loads:
            print(failed_loads)
    return loaded_midi_files, failed_loads

In [ ]:
loaded_midi_files, failed_midi_loads = load_midi_files(MIDI_FILE_PATH, PICKLING_PATH)

<font size=6>Mapping CSV data to MIDI files</font>

In [ ]:
def csv_data_to_songdata_list(csv_data: list[list[str]], 
                              failed_midi_load: dict[str, set[str]], 
                              midi_files_dict: dict[str, dict[str, PrettyMIDI]]) -> list[SongData]:
    song_data_list: list[SongData] = list()
    missing_midi_count = 0
    for row in csv_data:
        artist = row[0]
        title = row[1]
        if artist in failed_midi_load and title in failed_midi_load[artist]:
            print(f"Skipping {artist} - {title} due to previous MIDI load failure.")
            continue
        if artist in midi_files_dict and title in midi_files_dict[artist]:
            midi_file = midi_files_dict[artist][title]
            song_data = SongData(row, midi_file)
            song_data_list.append(song_data)
        else:
            missing_midi_count += 1
            print(f"Missing MIDI file for artist '{artist}' and title '{title}'")
    print(f"Total songs with missing MIDI files: {missing_midi_count}")
    return song_data_list

In [ ]:
train_midi_data: list[SongData] = csv_data_to_songdata_list(cleaned_lyric_train_data, failed_midi_loads, loaded_midi_files)
test_midi_data: list[SongData] = csv_data_to_songdata_list(cleaned_lyric_test_data, failed_midi_loads, loaded_midi_files)
print(f"Total training songs with MIDI data: {len(train_midi_data)}")
print(f"Total test songs with MIDI data: {len(test_midi_data)}")

<font size=6>Handling word embeddings</font>

Downloading pretrained word2vec, containing 300 dims, trained on news articles

In [ ]:
pretrained_word2vec = gensim.downloader.load('word2vec-google-news-300')

Extracting the vocabulary from the lyrics.
Getting the data from the test set aswell since the vocbulary needs to be known.

In [ ]:
lyrics_vocabulary: set[str] = set()
# Getting the data from the test set aswell since the vocbulary needs to be known
for song in train_midi_data + test_midi_data:
    for word in song.lyrics:
        lyrics_vocabulary.add(word)
print(f"Total unique words in lyrics vocabulary: {len(lyrics_vocabulary)}")

Creating unified embedding.
Extracting embeddings from word2vec and using random embeddings for words not found in word2vec.

In [ ]:
unified_embeddings: dict[str, np.ndarray] = dict()
existing_words_in_pretrained = 0
not_existing_in_pretrained = 0
added_stopwords = 0
for word in list(lyrics_vocabulary):
    if word in pretrained_word2vec:
        unified_embeddings[word] = pretrained_word2vec[word]
        existing_words_in_pretrained += 1
    else:
        unified_embeddings[word] = np.random.uniform(low=-1.0, high=1.0, size=(pretrained_word2vec.vector_size,)) # Random init for unknown words.  
        not_existing_in_pretrained += 1
    # Adding stopwords as well, since they are common and should be in the vocabulary.
for stopword in stopwords.words('english'):
    cleaned_stopword = re.sub(f"[{re.escape(string.punctuation)}]", " ", stopword.strip().lower()) # Cleaning the stopword, since it contains punctuation.
    if cleaned_stopword not in unified_embeddings:
        unified_embeddings[cleaned_stopword] = np.random.uniform(low=-1.0, high=1.0, size=(pretrained_word2vec.vector_size,))
        added_stopwords += 1
# Adding special tokens
unified_embeddings[EOL_STRING] = np.random.uniform(low=-1.0, high=1.0, size=(pretrained_word2vec.vector_size,))
unified_embeddings[UNK_STRING] = np.random.uniform(low=-1.0, high=1.0, size=(pretrained_word2vec.vector_size,))
unified_embeddings[EOS_STRING] = np.random.uniform(low=-1.0, high=1.0, size=(pretrained_word2vec.vector_size,))

print(f"Total unique words in lyrics vocabulary: {len(lyrics_vocabulary)}")
print(f"Existing words in pretrained embeddings: {existing_words_in_pretrained}")
print(f"Not existing in pretrained embeddings (randomly initialized): {not_existing_in_pretrained}")
print(f"Added stopwords (randomly initialized): {added_stopwords}")

<font size=6>Handling artist embeddings</font>

Using simple indexing for it.

In [ ]:
train_artists = [song.artist for song in train_midi_data]
test_artists = [song.artist for song in test_midi_data]
artist_set: set = (set(train_artists).union(set(test_artists)))
artist_to_index: dict[str, int] = dict()
index_to_artist: dict[int, str] = dict()
for index, artist in enumerate(artist_set):
    artist_to_index[artist] = index
    index_to_artist[index] = artist

<font size=6>Model 1: Simple concatenation</font>

In [ ]:
# Integration Method 1: Simple Concatenation - Melody features are concatenated to each word embedding
class LyricsGenerator_Concatenation(nn.Module):
  def __init__(self, 
               vocab_size: int, 
               input_size: int, 
               hidden_layer_dim: int, 
               size_of_midi_features: int = NUMBER_OF_EXTRACT_MIDI_FEATURES,
               size_of_word_embeddings: int = WORD_EMBEDDING_SIZE,
               num_layers: int = LSTM_LAYERS, 
               dropout_rate: float = DROPOUT
               ):
    super(LyricsGenerator_Concatenation, self).__init__()
    self.vocab_size: int = vocab_size
    self.num_layers: int = num_layers
    self.word_embedding_size: int = size_of_word_embeddings
    self.size_of_midi_features: int = size_of_midi_features
    self.lstm = nn.LSTM(input_size, hidden_layer_dim, num_layers,
                        batch_first=True, dropout=dropout_rate if num_layers > 1 else 0)

    self.dropout = nn.Dropout(dropout_rate)
    self.fc = nn.Linear(hidden_layer_dim, vocab_size)

  def forward(self, x):
    lstm_out, _ = self.lstm(x)
    output_post_dropout = self.dropout(lstm_out)
    logits = self.fc(output_post_dropout)
    return logits

<font size=6>Model 2: Attention</font>

In [ ]:
class LyricsGenerator_Attention(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 hidden_layer_dim: int = HIDDEN_LAYER_DIM_ATTENTION,
                 size_of_word_embeddings: int = WORD_EMBEDDING_SIZE,
                 size_of_midi_features: int = NUMBER_OF_EXTRACT_MIDI_FEATURES,
                 size_of_artist_index_dim: int = SIZE_OF_ARTIST_INDEX,
                 num_layers: int = 1,
                 dropout_rate: float = 0.1):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = size_of_word_embeddings
        self.melody_dim = size_of_midi_features
        self.artist_dim = size_of_artist_index_dim
        self.hidden = hidden_layer_dim
        self.size_of_artist_index_dim = size_of_artist_index_dim
        self.hidden_layer_dim = hidden_layer_dim

        # Separate encoders
        self.word_rnn = nn.LSTM(self.embedding_dim, self.hidden_layer_dim, num_layers,
                                batch_first=True,
                                dropout=dropout_rate if num_layers > 1 else 0)

        # MIDI & Artist: static per song -> pool to a single token then project
        self.midi_proj   = nn.Linear(self.melody_dim, self.hidden_layer_dim, bias=True)
        self.artist_proj = nn.Linear(self.artist_dim, self.hidden_layer_dim, bias=True)

        # Per-modality attention projections (keys/values); queries shared
        self.k_word = nn.Linear(self.hidden_layer_dim, self.hidden_layer_dim, bias=False)
        self.v_word = nn.Linear(self.hidden_layer_dim, self.hidden_layer_dim, bias=False)

        self.k_midi = nn.Linear(self.hidden_layer_dim, self.hidden_layer_dim, bias=False)    # operates on pooled MIDI token
        self.v_midi = nn.Linear(self.hidden_layer_dim, self.hidden_layer_dim, bias=False)

        self.k_artist = nn.Linear(self.hidden_layer_dim, self.hidden_layer_dim, bias=False)  # operates on pooled Artist token
        self.v_artist = nn.Linear(self.hidden_layer_dim, self.hidden_layer_dim, bias=False)

        # Shared query from global summary (word_last | midi_tok | artist_tok)
        self.q_proj = nn.Linear(3 * self.hidden_layer_dim, self.hidden_layer_dim, bias=False)

        # Late fusion over 3 heads
        self.fusion_scorer = nn.Linear(3 * self.hidden_layer_dim, 3)

        # Per-step fusion of [ctx || base] used by attn_seq_logits
        self.combine_proj_seq = nn.Linear(2 * self.hidden_layer_dim, self.hidden_layer_dim)

        # Output head
        self.combine_proj = nn.Linear(self.hidden_layer_dim + 3 * self.hidden_layer_dim, self.hidden_layer_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(self.hidden_layer_dim, vocab_size)

    def forward(self, x: torch.Tensor):
        """
        x: [B, T, E+M+A] = [word_emb || midi || artist]
        returns:
          logits: [B, V]
        """
        x = x.float()
        _, _, features = x.shape
        E, M, A, H = self.embedding_dim, self.melody_dim, self.artist_dim, self.hidden
        assert features == (E + M + A)

        # Split
        word = x[..., :E]               # [B,T,E]
        midi = x[..., E:E+M]            # [B,T,M] (likely static over T)
        art  = x[..., E+M:E+M+A]        # [B,T,A] (likely static over T)

        # Encoders
        word_out, (word_last, _) = self.word_rnn(word)   # word_out: [B,T,H]
        word_last = word_last[-1]                        # [B,H]

        # Pool static modalities to a single token per song
        midi_tok   = self.midi_proj(midi.mean(dim=1))    # [B,H]
        artist_tok = self.artist_proj(art.mean(dim=1))   # [B,H]

        # Shared query
        global_last = torch.cat([word_last, midi_tok, artist_tok], dim=-1)  # [B,3H]
        q = self.q_proj(global_last).unsqueeze(1)                            # [B,1,H]

        # --- Head 1: WORD (temporal over T) ---
        Kw = self.k_word(word_out)                               # [B,T,H]
        Vw = self.v_word(word_out)                               # [B,T,H]
        scores_w = torch.bmm(q, Kw.transpose(1, 2)).squeeze(1) / math.sqrt(H)  # [B,T]
        attn_w = F.softmax(scores_w, dim=-1)                     # [B,T]

        ctx_w = torch.bmm(attn_w.unsqueeze(1), Vw).squeeze(1)    # [B,H]
        # --- Head 2: MIDI (single token) ---
        Vm = self.v_midi(midi_tok).unsqueeze(1)                  # [B,1,H]
        ctx_m = Vm.squeeze(1)                                     # [B,H]

        # --- Head 3: ARTIST (single token) ---
        Va = self.v_artist(artist_tok).unsqueeze(1)              # [B,1,H]
        ctx_a = Va.squeeze(1)                                     # [B,H]

        # Late fusion across heads
        ctx_cat = torch.cat([ctx_w, ctx_m, ctx_a], dim=-1)          # [B,3H]
        alpha = F.softmax(self.fusion_scorer(ctx_cat), dim=-1)       # [B,3]
        w_w, w_m, w_a = alpha.unbind(dim=-1)                         # each [B]
        fused = (w_w.unsqueeze(-1)*ctx_w +
                 w_m.unsqueeze(-1)*ctx_m +
                 w_a.unsqueeze(-1)*ctx_a)                            # [B,H]

        combined = torch.tanh(self.combine_proj(torch.cat([fused, global_last], dim=-1)))  # [B,H]
        logits = self.fc(self.dropout(combined))                     # [B,V]

        return logits


<font size=6>Running the models</font>

Simple word-to-index and index-to-word for word generation afterwards

In [ ]:
word_to_id = {word_in_vocab: index_of_word for index_of_word, word_in_vocab in enumerate(unified_embeddings.keys())} 
id_to_word = {index_of_word: word_in_vocab for word_in_vocab, index_of_word in word_to_id.items()}

<font size=4>Preparing dataset and dataloaders</font>

In [ ]:
songdata_train_dataset_harmony_midi_features = SongDataset(train_midi_data, unified_embeddings, artist_to_index, word_to_id, HARMONY_FEATURES)
songdata_train_dataset_rhythm_midi_features = SongDataset(train_midi_data, unified_embeddings, artist_to_index, word_to_id, RHYTHM_FEATURES)

songdata_test_dataset_harmony_midi_features = SongDataset(train_midi_data, unified_embeddings, artist_to_index, word_to_id, HARMONY_FEATURES)
songdata_test_dataset_rhythm_midi_features = SongDataset(train_midi_data, unified_embeddings, artist_to_index, word_to_id, RHYTHM_FEATURES)

In [ ]:
# ------------------------------ HARMONY ------------------------------
training_set_harmony, validation_set_harmony = train_test_split(songdata_train_dataset_harmony_midi_features, test_size=VALIDATION_SPLIT, random_state=RANDOM_LOADER_SEED, shuffle=True)
training_data_loader_harmony = data.DataLoader(training_set_harmony, batch_size=BATCH_SIZE, shuffle=True)

validation_data_loader_harmony = data.DataLoader(validation_set_harmony, batch_size=BATCH_SIZE, shuffle=False)
test_data_loader_harmony = data.DataLoader(songdata_test_dataset_harmony_midi_features, batch_size=5, shuffle=False) # That's the size of the test set.

# ------------------------------ RHYTHM ------------------------------
training_set_rhythm, validation_set_rhythm = train_test_split(songdata_train_dataset_rhythm_midi_features, test_size=VALIDATION_SPLIT, random_state=RANDOM_LOADER_SEED, shuffle=True)
training_data_loader_rhythm = data.DataLoader(training_set_rhythm, batch_size=BATCH_SIZE, shuffle=True)

validation_data_loader_rhythm = data.DataLoader(validation_set_rhythm, batch_size=BATCH_SIZE, shuffle=False)
test_data_loader_rhythm = data.DataLoader(songdata_test_dataset_rhythm_midi_features, batch_size=5, shuffle=False) # That's the size of the test set.

Helper function for the attention model.

In [ ]:
def attn_seq_logits(model: nn.Module, x: torch.Tensor) -> torch.Tensor:
    # x: [B,T,E+M+A] -> logits [B,T-1,V]
    x = x.float()
    B, T, _ = x.shape
    E, M, A, H = model.embedding_dim, model.melody_dim, model.artist_dim, model.hidden_layer_dim

    # split
    word   = x[..., :E]            # [B,T,E]
    midi   = x[..., E:E+M]         # [B,T,M]
    artist = x[..., E+M:E+M+A]     # [B,T,A]

    # word encoder
    rnn_out, (h_n, _) = model.word_rnn(word)     # rnn_out: [B,T,H]
    h_last = h_n[-1]                              # [B,H]

    # pooled static tokens
    midi_tok   = model.midi_proj(midi.mean(1))    # [B,H]
    artist_tok = model.artist_proj(artist.mean(1))# [B,H]

    # query from global summary
    global_last = torch.cat([h_last, midi_tok, artist_tok], dim=-1)  # [B,3H]
    q = model.q_proj(global_last).unsqueeze(1)                       # [B,1,H]

    # attention over word timeline
    K = model.k_word(rnn_out)                         # [B,T,H]
    V = model.v_word(rnn_out)                         # [B,T,H]
    scores = torch.bmm(q, K.transpose(1, 2)).squeeze(1) / math.sqrt(H)  # [B,T]
    attn = torch.softmax(scores, dim=-1)              # [B,T]
    context = torch.bmm(attn.unsqueeze(1), V).squeeze(1)                # [B,H]

    # per-step combine with base hidden states
    base = rnn_out[:, :-1, :]                         # [B,T-1,H]
    ctx  = context.unsqueeze(1).expand(-1, base.size(1), -1)  # [B,T-1,H]
    combined = torch.tanh(model.combine_proj_seq(torch.cat([ctx, base], dim=-1)))  # [B,T-1,H]
    logits = model.fc(model.dropout(combined))        # [B,T-1,V]
    return logits

In [ ]:
def seq_logits(model, model_input):  # returns [B, T-1, V] for both model types
    if isinstance(model, LyricsGenerator_Attention):
        return attn_seq_logits(model, model_input)               # [B,T-1,V]
    elif isinstance(model, LyricsGenerator_Concatenation):
        return model(model_input)[:, :-1, :]               # [B,T-1,V] from [B,T,V]
    else:
        raise ValueError(f'Model not recognized: {model}')

In [ ]:
def train_model(model: nn.Module,
                train_loader: data.DataLoader,
                val_loader: data.DataLoader,
                test_loader: data.DataLoader,
                num_epochs: int = MAX_EPOCHS,
                learning_rate: float = LEARNING_RATE,
                patiance_factor: float = PATIENCE_FACTOR,
                patiance_epochs: int = PATIENCE_EPOCHS,
                run_name: Optional[str] = None,
                save_path: str = ""):
    run_name = run_name or model.__class__.__name__
    writer = SummaryWriter(log_dir=f"runs/{run_name}")  # <- unique folder per model
    writer.add_text("run/model_name", model.__class__.__name__, global_step=0)
    model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patiance_epochs)
    best_model_state_dict = copy.deepcopy(model.state_dict())
    train_losses: list[float] = list()
    val_losses: list[float] = list()
    best_validation_loss: float = 10000.0
    epochs_with_no_improvements: int = 0
    vocabulary_size = model.vocab_size
    
    for epoch in range(num_epochs):
        current_time = time.time()
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            inputs = inputs.to(device=device, dtype=torch.float32)
            targets = targets.to(device=device, dtype=torch.long)
            optimizer.zero_grad()
            logits = seq_logits(model, inputs)
            loss = criterion(
                logits.contiguous().view(-1, vocabulary_size),
                targets[:, 1:].contiguous().view(-1)
            )
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        validation_running_loss = 0.0
        test_running_loss = 0.0
        model.eval()
        with torch.no_grad():
            for val_inputs, val_targets in val_loader:
                val_inputs  = val_inputs.to(device=device, dtype=torch.float32)
                val_targets = val_targets.to(device=device, dtype=torch.long)
                val_logits = seq_logits(model, val_inputs)         # [B,T-1,V]
                val_loss = criterion(
                    val_logits.contiguous().view(-1, vocabulary_size),
                    val_targets[:, 1:].contiguous().view(-1)
                )
                validation_running_loss += val_loss.item() * val_inputs.size(0)
            val_epoch_loss = validation_running_loss / len(val_loader.dataset)
            val_losses.append(val_epoch_loss)
            writer.add_scalar("Loss/val", val_epoch_loss, epoch)   # <-- add here
        scheduler.step(val_epoch_loss)
        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)
        writer.add_scalar("Loss/train", epoch_loss, epoch)  # TensorBoard
        test_epoch_loss = test_running_loss / len(test_loader.dataset)
        writer.add_scalar("Loss/test", test_epoch_loss, epoch)
        if  best_validation_loss - val_epoch_loss >= patiance_factor:
            epochs_with_no_improvements = 0
            best_validation_loss = val_epoch_loss
            best_model_state_dict = copy.deepcopy(model.state_dict())
        else:
            epochs_with_no_improvements += 1
            print(f'No improvement in epoch. Patiance: {epochs_with_no_improvements}\\{patiance_epochs}')
        finish_time = time.time() - current_time
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_epoch_loss:.4f}, time: {finish_time}')
        if epochs_with_no_improvements >= patiance_epochs:
            print(f'Training ended prematurely due to lack of improvement.')
            break
    writer.close()  # Close TensorBoard writer
    model.load_state_dict(best_model_state_dict)
    model.eval()
    torch.save(model.state_dict(), os.path.join(save_path, f"{run_name}_{num_epochs}.pth"))
    return model, train_losses, val_losses


<font size=4>Running model 1: Concatenation with midi features: Harmony</font>

In [ ]:
model_concatenation_harmony = LyricsGenerator_Concatenation(
    vocab_size=len(songdata_train_dataset_harmony_midi_features.word_embeddings),
    input_size=pretrained_word2vec.vector_size + NUMBER_OF_EXTRACT_MIDI_FEATURES + 1, # word embedding + melody features + artist index
    hidden_layer_dim=HIDDEN_LAYER_DIM,
    dropout_rate=DROPOUT,
    num_layers=LSTM_LAYERS,
)
print(f'LyricsGenerator_Concatenation harmony:')
summary(model_concatenation_harmony, input_size=(BATCH_SIZE, SEQUENCE_LENGTH, WORD_EMBEDDING_SIZE + NUMBER_OF_EXTRACT_MIDI_FEATURES + SIZE_OF_ARTIST_INDEX), dtypes=[torch.float32])

In [ ]:
model_harmony, harmony_concatenation_training_loss, harmony_concatenation_validation_loss = train_model(model_concatenation_harmony,
            training_data_loader_harmony, 
            validation_data_loader_harmony, 
            test_data_loader_harmony,
            run_name=f"LyricsGenerator_Concatenation_Harmony_{PATIENCE_FACTOR}"
            )

<font size=4>Running model 1: Concatenation with midi features: Rhythm</font>

In [ ]:
model_concatenation_rhythm = LyricsGenerator_Concatenation(
    vocab_size=len(songdata_train_dataset_rhythm_midi_features.word_embeddings),
    input_size=pretrained_word2vec.vector_size + NUMBER_OF_EXTRACT_MIDI_FEATURES + 1, # word embedding + melody features + artist index
    hidden_layer_dim=HIDDEN_LAYER_DIM,
    num_layers=LSTM_LAYERS,
    dropout_rate=DROPOUT,
)
print(f'LyricsGenerator_Concatenation rhythm:')
summary(model_concatenation_rhythm, input_size=(BATCH_SIZE, SEQUENCE_LENGTH, WORD_EMBEDDING_SIZE + NUMBER_OF_EXTRACT_MIDI_FEATURES + SIZE_OF_ARTIST_INDEX), dtypes=[torch.float32])

In [ ]:
model_concatenation_rhythm, rhythm_concatenation_training_loss, rhythm_concatenation_validation_loss = train_model(model_concatenation_rhythm,
            training_data_loader_rhythm, 
            validation_data_loader_rhythm, 
            test_data_loader_rhythm,
            run_name=f"LyricsGenerator_Concatenation_Rhythm_{PATIENCE_FACTOR}"
            )


<font size=4>Running model 2: Attention with midi features: Harmony</font>

In [ ]:
attention_model_harmony = LyricsGenerator_Attention(
    vocab_size=len(songdata_train_dataset_harmony_midi_features.word_embeddings),
    size_of_word_embeddings=WORD_EMBEDDING_SIZE,
    size_of_midi_features=NUMBER_OF_EXTRACT_MIDI_FEATURES,
    size_of_artist_index_dim=SIZE_OF_ARTIST_INDEX,
    hidden_layer_dim=HIDDEN_LAYER_DIM_ATTENTION,
)
print(f'LyricsGenerator_Attention Harmony:')
summary(attention_model_harmony, 
        input_size=(BATCH_SIZE, SEQUENCE_LENGTH, WORD_EMBEDDING_SIZE + NUMBER_OF_EXTRACT_MIDI_FEATURES + SIZE_OF_ARTIST_INDEX), 
        dtypes=[torch.float32])

In [ ]:
attention_model_harmony, attention_harmony_training_loss, attention_harmony_validation_loss = train_model(attention_model_harmony,
            training_data_loader_harmony, 
            validation_data_loader_harmony, 
            test_data_loader_harmony,
            run_name=f"LyricsGenerator_Attention_Harmony_{PATIENCE_FACTOR}"
            )

<font size=4>Running model 2: Attention with midi features: Rhythm</font>

In [ ]:
attention_model_rhythm = LyricsGenerator_Attention(
    vocab_size=len(songdata_test_dataset_rhythm_midi_features.word_embeddings),
    size_of_word_embeddings=WORD_EMBEDDING_SIZE,
    size_of_midi_features=NUMBER_OF_EXTRACT_MIDI_FEATURES,
    size_of_artist_index_dim=SIZE_OF_ARTIST_INDEX,
    hidden_layer_dim=HIDDEN_LAYER_DIM_ATTENTION,
)
print(f'LyricsGenerator_Attention Rhythm:')
summary(attention_model_rhythm, 
        input_size=(BATCH_SIZE, SEQUENCE_LENGTH, WORD_EMBEDDING_SIZE + NUMBER_OF_EXTRACT_MIDI_FEATURES + SIZE_OF_ARTIST_INDEX), 
        dtypes=[torch.float32])

In [ ]:
attention_model_rhythm, attention_rhythm_training_loss, attention_rhythm_validation_loss = train_model(attention_model_rhythm,
            training_data_loader_rhythm, 
            validation_data_loader_rhythm, 
            test_data_loader_rhythm,
            run_name=f"LyricsGenerator_Attention_Rhythm_{PATIENCE_FACTOR}"
            )


<font size=6> Displaying graph comparing for each model and combining results </font>

In [ ]:
def display_graphs_for_loss(training_loss_per_epoch: list[float], validation_loss_per_epoch: list[float], subtitle: str =""):
    plt.title("Training and validation loss over epochs")
    plt.suptitle(subtitle)
    plt.plot(training_loss_per_epoch[:], label="Training loss")
    plt.plot(validation_loss_per_epoch[:], label="Validation loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
def display_graph_comparing_each_model(title: str, data_points: list[tuple[str, list[float]]]): # title, list(data_point_name, data_points)
    figure, axes = plt.subplots()
    for name, data_points_per_model in data_points:
        data_point_list = range(1, len(data_points_per_model) + 1)
        axes.plot(data_point_list, data_points_per_model, label=name)

    axes.set_xlabel('Epochs')
    axes.set_ylabel('Loss')
    axes.set_title(title)
    axes.legend(title="Models", loc="upper right")
    axes.grid(True, linestyle="--", alpha=0.4)
    plt.show()

In [ ]:
display_graphs_for_loss(harmony_concatenation_training_loss, harmony_concatenation_validation_loss, subtitle="Concatenation + Harmony")
display_graphs_for_loss(rhythm_concatenation_training_loss, rhythm_concatenation_validation_loss, subtitle="Concatenation + Rhythm")
display_graphs_for_loss(attention_harmony_training_loss, attention_harmony_validation_loss, subtitle="Attention + Harmony")
display_graphs_for_loss(attention_rhythm_training_loss, attention_rhythm_validation_loss, subtitle="Attention + Rhythm")

Comparing models against each other

In [ ]:
general_training_loss = [("Concatenation + Harmony", harmony_concatenation_training_loss),
 ("Concatenation + Rhythm", rhythm_concatenation_training_loss),
 ("Attention + Harmony", attention_harmony_training_loss),
 ("Attention + Rhythm", attention_rhythm_training_loss)]

general_validation_loss = [("Concatenation + Harmony", harmony_concatenation_validation_loss),
 ("Concatenation + Rhythm", rhythm_concatenation_validation_loss),
 ("Attention + Harmony", attention_harmony_validation_loss),
 ("Attention + Rhythm", attention_rhythm_validation_loss)]

In [ ]:
display_graph_comparing_each_model("Training_loss_for_each_model", general_training_loss)
display_graph_comparing_each_model("Validation_loss_for_each_model", general_training_loss)

<font size=6>Generating Lyrics</font>

A function that returns the k most likely words given the input, used for coherent lyric generatin.

In [ ]:
@torch.no_grad()
def predict_next_word(
    model: nn.Module,
    word_sequence: list[str],
    artist_index: int,
    melody_vec,
    word_to_id: dict[str, int],
    id_to_word: dict[int, str],
    embedding_weight: torch.Tensor,
    device: str = "cpu",
    forbidden_words: list[str] | None = None,
    strengthened_words: list[tuple[str, float]] | None = None,
):
    model = model.to(device).eval()

    # embedding matrix on device
    if embedding_weight.device.type != device:
        embedding_weight = embedding_weight.to(device)

    # melody -> [M]
    melody_vec = torch.as_tensor(melody_vec, dtype=torch.float32, device=device)

    # seq -> ids -> embeddings
    unk_id = word_to_id.get(UNK_STRING, UNK_ID)
    seq_ids = torch.tensor([word_to_id.get(w, unk_id) for w in word_sequence],
                           device=device, dtype=torch.long)              # [T]
    seq_embs = embedding_weight[seq_ids]                                  # [T, E]

    T = seq_embs.size(0)
    # broadcast melody + artist
    melody_broadcast = melody_vec.expand(T, -1)                           # [T, M]
    # NOTE: your model’s artist dim must equal size_of_artist_index_dim; here we use 1
    artist_broadcast = torch.full((T, 1), float(artist_index),
                                  dtype=torch.float32, device=device)     # [T, 1]

    # concat → [1, T, E+M+artist_dim]
    x = torch.cat([seq_embs, melody_broadcast, artist_broadcast], dim=1).unsqueeze(0)  # [1,T,D]

    # forward
    out = model(x)
    if isinstance(model, LyricsGenerator_Attention):
        logits = out[0]         # attention model: [1, V]
    elif isinstance(model, LyricsGenerator_Concatenation):
        logits = out[:, -1, :]  # [1, V]
    else:
        raise ValueError(f"Model: {model} is not recognized.")
    
    logits = logits.squeeze(0)  # [V]
    # forbid
    if forbidden_words:
        forb_idx = [word_to_id[w] for w in forbidden_words if w in word_to_id]
        if forb_idx:
            logits[torch.tensor(forb_idx, device=device, dtype=torch.long)] = float("-inf")

    probs = torch.softmax(logits, dim=-1)  # [V]

    # strengthen: set explicit probabilities for selected words
    if strengthened_words:
        pairs = [(w, float(p)) for (w, p) in strengthened_words if (w in word_to_id) and (p >= 0.0)]
        if pairs:
            idxs = torch.tensor([word_to_id[w] for (w, _) in pairs], device=device, dtype=torch.long)
            p_desired = torch.tensor([min(p, 1.0) for (_, p) in pairs], device=device, dtype=probs.dtype)
            # aggregate dupes
            uniq, inv = torch.unique(idxs, return_inverse=True)
            p_agg = torch.zeros_like(uniq, dtype=probs.dtype).scatter_add(0, inv, p_desired)

            base = probs.clone()
            base[uniq] = 0.0
            base_sum = base.sum()
            sum_p = p_agg.sum()

            if float(sum_p) >= 1.0 - 1e-8 or base_sum <= 1e-12:
                probs = torch.zeros_like(probs)
                probs[uniq] = p_agg / (sum_p + 1e-12)
            else:
                remain = 1.0 - float(sum_p)
                base = base * (remain / (base_sum + 1e-12))
                probs = base
                probs[uniq] = p_agg

    idx = torch.multinomial(probs, num_samples=1, replacement=True)
    return id_to_word[int(idx.item())]

Printing the generated text and handling tokens

In [ ]:
def print_generated_lyrics(generated_lyrics: list[str]):
    capitalize = True
    for word in generated_lyrics:
        if word == EOL_STRING:
            capitalize = True
            print()
        if word == EOS_STRING:
            break
        if word != EOL_STRING:
            if capitalize:
                capitalize = False
                print(word.title(), end=' ')
            else:
                print(word, end=' ')
    print()

Generating the lyrics and maintaining the lyrics generated

In [ ]:
def generate_lyrics(
        model_to_use: nn.Module,
        model_title: str,
        initial_word: str,
        melody_features: np.ndarray,
        melody_title: str,
        artist_to_use: str,
        word_to_id: dict[str, int],
        id_to_word: dict[int, str],
        artist_to_index: dict[str, int],
        word_embeddings: dict[str, np.ndarray],
        max_song_length: int = MAX_SONG_LENGTH_WORDS,
        sequence_length: int = SEQUENCE_LENGTH,
        device: str = "cuda" if torch.cuda.is_available() else "cpu"
):
    """
    Generates lyrics word by word using the model, melody, and artist.
    Picks next word randomly from top_k candidates according to their normalized probabilities.
    Artificially increases probability of EOS_STRING after half of max_song_length.
    Prints the generated lyrics with line breaks at <eol>.
    Enforces some more grammatical rules.
    Returns: generated_lyrics (list of str), artist, melody_title.
    """
    print(f"------------------------- Model: {model_title} -------------------------")
    print(f"Generating song")
    print(f"--------------------------------------------------")
    print(f"Initial Word: {initial_word}")
    print(f"Artist: {artist_to_use}")
    print(f"Melody: {melody_title}")
    print(f"--------------------------------------------------")
    initial_word = initial_word.lower()
    melody_vec = torch.as_tensor(melody_features[0], dtype=torch.float32, device=device)
    artist_idx = artist_to_index.get(artist_to_use.lower(), -1)
    if artist_idx == -1:
        print(f"Warning: Artist '{artist_to_use}' not found, using index -1.")

    embedding_weight = torch.from_numpy(
        np.stack([word_embeddings[w].astype(np.float32) for w in word_to_id], axis=0)
    ).to(device)
    context: deque = deque()
    context.extendleft([UNK_STRING for _ in range(sequence_length - 1)])
    context.appendleft(initial_word)
    unk_index: int = 1
    generated_lyrics = [initial_word]
    words_in_song: int = 0
    current_word: str = initial_word
    minimum_song_length = int(max_song_length / 2)
    current_words_in_line: int = 1
    current_word = ""
    next_word = ""
    words_not_to_end_lines_on: list[str] = ['the', 'at', 'and']
    number_of_lines_in_song: int = 0
    while True:
        # Tries to enforce certain rules.
        # Don't allow end of song before minimum amount of lines.
        # Don't repeat the same word twice
        # Don't allow lines that are too short.
        # Don't end lines on words in a way that would make no sense.
        forbidden_words = [current_word]
        strengthened_words = list()
        if words_in_song < int(minimum_song_length):
            forbidden_words.append(EOS_STRING)
        if current_word in words_not_to_end_lines_on:
            forbidden_words.extend([EOL_STRING, EOS_STRING])
        if current_words_in_line < MIN_LINE_LENGTH:
            forbidden_words.append(EOL_STRING)
        if current_words_in_line > int(MIN_LINE_LENGTH):
            probability_of_eol: float = min((current_words_in_line - int(MIN_LINE_LENGTH))/ int(MIN_LINE_LENGTH), 1.0)
            strengthened_words.append((EOL_STRING, probability_of_eol))
        if words_in_song > int(MAX_SONG_LENGTH_WORDS / 2):
            probability_of_eos: float = min((words_in_song - int(MAX_SONG_LENGTH_WORDS / 2))/ int(MAX_SONG_LENGTH_WORDS/2), 1.0)
            strengthened_words.append((EOS_STRING, probability_of_eos))
        next_word = predict_next_word(
            model=model_to_use,
            word_sequence=context,
            artist_index=artist_idx,
            melody_vec=melody_vec,
            word_to_id=word_to_id,
            id_to_word=id_to_word,
            embedding_weight=embedding_weight,
            forbidden_words=forbidden_words,
            strengthened_words=strengthened_words,
            device=device,
        )
        if next_word == EOS_STRING and current_word == EOL_STRING:
            generated_lyrics[-1] = next_word # In case an end of song comes after linebreak, just end the song instead.
        else:
            generated_lyrics.append(next_word)
        current_word = next_word
        if unk_index < sequence_length:
            context[unk_index] = next_word
            unk_index += 1
        else:
            context.popleft()
            context.append(next_word)
        if next_word == EOS_STRING:
            break
        if words_in_song >= max_song_length:
            generated_lyrics.append(EOS_STRING)
            break
        if next_word != EOL_STRING:
            words_in_song += 1
            current_words_in_line += 1
        else:
            number_of_lines_in_song += 1
            current_words_in_line = 0
    print_generated_lyrics(generated_lyrics=generated_lyrics)
    print(f"--------------------------------------------------")
    print("\n--- End of generated lyrics ---")
    print(f"--------------------------------------------------")
    print(f'Number of words in lyrics: {words_in_song}')
    print(f'Number of lines in lyrics: {number_of_lines_in_song}')
    print(f"--------------------------------------------------")
    return generated_lyrics, artist_to_use, melody_title

<font size=6>Testing with the testing set</font>

For each melody and each model, the output of the architecture given the melody and the initial word of the real lyrics.

In [ ]:
harmony_model_name_model_list: list[tuple[str, nn.Module]] = [
    ("Concatenation + Harmony", model_concatenation_harmony),
    ("Attention + Harmony", attention_model_harmony)
]
rhythm_model_name_model_list: list[tuple[str, nn.Module]] = [
    ("Concatenation + Rhythm", model_concatenation_rhythm),
    ("Attention + Rhythm", attention_model_rhythm)
]

In [ ]:
for model_name, model in harmony_model_name_model_list:
    print('======================================')
    print('===========Harmoney Models============')
    print('======================================')
    for test_song in test_midi_data:
        test_song: SongData
        lyrics, artist, melody = generate_lyrics(
            model_to_use=model,
            model_title=model_name,
            initial_word=test_song.lyrics[0],
            melody_features=test_song.midi_harmony_and_pitch_features,
            melody_title=test_song.title,
            artist_to_use=test_song.artist,
            word_to_id=word_to_id,
            id_to_word=id_to_word,
            artist_to_index=artist_to_index,
            word_embeddings=unified_embeddings,
            max_song_length=len([word for word in test_song.lyrics if word != EOL_STRING])
        )
    print('======================================')


In [ ]:
for model_name, model in rhythm_model_name_model_list:
    print('==================================================================================')
    print('==================================Rhythm Models===================================')
    print('==================================================================================')
    for test_song in test_midi_data:
        test_song: SongData
        lyrics, artist, melody = generate_lyrics(
            model_to_use=model,
            model_title=model_name,
            initial_word=test_song.lyrics[0],
            melody_features=test_song.midi_rhythm_and_dynamic_features,
            melody_title=test_song.title,
            artist_to_use=test_song.artist,
            word_to_id=word_to_id,
            id_to_word=id_to_word,
            artist_to_index=artist_to_index,
            word_embeddings=unified_embeddings,
            max_song_length=len([word for word in test_song.lyrics if word != EOL_STRING])
        )

For each melody, the output of the architecture given the melody and different starting words. The same word should be used for all melodies.

In [ ]:
starting_words: list[str] = ['love', 'baby', 'time']
for model_name, model in harmony_model_name_model_list:
    print('======================================')
    print('===========Harmoney Models============')
    print('======================================')
    for word in starting_words:
        print(f'-------------Initial Word Selected: {word}-------------------')
        for test_song in test_midi_data:
            print('--------------------------------')
            test_song: SongData
            lyrics, artist, melody = generate_lyrics(
                model_to_use=model,
                model_title=model_name,
                initial_word=word,
                melody_features=test_song.midi_harmony_and_pitch_features,
                melody_title=test_song.title,
                artist_to_use=test_song.artist,
                word_to_id=word_to_id,
                id_to_word=id_to_word,
                artist_to_index=artist_to_index,
                word_embeddings=unified_embeddings,
                max_song_length=len([word for word in test_song.lyrics if word != EOL_STRING])
            )
            print('--------------------------------')
    print('======================================')

In [ ]:
starting_words: list[str] = ['love', 'baby', 'time']
for model_name, model in rhythm_model_name_model_list:
    print('======================================')
    print('============Rhythm Models=============')
    print('======================================')
    for word in starting_words:
        print(f'-------------Initial Word Selected: {word}-------------------')
        for test_song in test_midi_data:
            print('--------------------------------')
            test_song: SongData
            lyrics, artist, melody = generate_lyrics(
                model_to_use=model,
                model_title=model_name,
                initial_word=word,
                melody_features=test_song.midi_rhythm_and_dynamic_features,
                melody_title=test_song.title,
                artist_to_use=test_song.artist,
                word_to_id=word_to_id,
                id_to_word=id_to_word,
                artist_to_index=artist_to_index,
                word_embeddings=unified_embeddings,
                max_song_length=len([word for word in test_song.lyrics if word != EOL_STRING])
            )
            print('--------------------------------')
    print('======================================')